In [1]:
import pandas as pd
import numpy as np
import random
import sys

sys.path.append('./ex_fuzzy/')
sys.path.append('../ex_fuzzy/')

from sklearn import datasets
from sklearn.model_selection import train_test_split

import ex_fuzzy.fuzzy_sets as fs
import ex_fuzzy.evolutionary_fit as GA
import ex_fuzzy.utils as utils
import ex_fuzzy.eval_tools as eval_tools

df = pd.read_csv('./paygap data/heart_attack.csv')
df

,age,gender,region,income_level,hypertension,diabetes,cholesterol_level,obesity,waist_circumference,family_history,...,blood_pressure_diastolic,fasting_blood_sugar,cholesterol_hdl,cholesterol_ldl,triglycerides,EKG_results,previous_heart_disease,medication_usage,participated_in_free_screening,heart_attack
0,60,Male,Rural,Middle,0,1,211,0,83,0,...,62,173,48,121,101,Normal,0,0,0,0
1,53,Female,Urban,Low,0,0,208,0,106,1,...,76,70,58,83,138,Normal,1,0,1,0
2,62,Female,Urban,Low,0,0,231,1,112,1,...,74,118,69,130,171,Abnormal,0,1,0,1
3,73,Male,Urban,Low,1,0,202,0,82,1,...,65,98,52,85,146,Normal,0,1,1,0
4,52,Male,Urban,Middle,1,0,232,0,89,0,...,75,104,59,127,139,Normal,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158350,60,Female,Rural,Middle,0,0,199,0,79,0,...,74,79,52,238,234,Normal,0,1,0,0
158351,64,Female,Urban,Low,0,0,179,0,93,0,...,73,111,41,177,239,Normal,0,1,1,1
158352,30,Male,Urban,Middle,0,0,220,0,78,0,...,87,127,50,158,201,Abnormal,1,1,1,1
158353,74,Male,Urban,Middle,0,0,180,0,58,1,...,70,119,52,153,171,Normal,1,0,1,1


In [2]:
X = df.drop(columns=['heart_attack'])
y = df['heart_attack']

missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])
df['alcohol_consumption'] = df['alcohol_consumption'].fillna('Unknown')

alcohol_consumption    94848
dtype: int64


In [3]:
fz_type_studied = fs.FUZZY_SETS.t1  # T1 fuzzy sets
n_linguistic_variables = 3  # Define the number of linguistic variables
n_gen = 30
n_pop = 30
n_rules = 20
categorical_mask = np.zeros(X.shape[1], dtype=int)
for i, column in enumerate(X.columns):
    if X[column].dtype == 'object':
        _, unique_classes = pd.factorize(X[column])
        categorical_mask[i] = len(unique_classes)
        print(f"Column '{column}' unique classes: {unique_classes.tolist()}")


precomputed_partitions = utils.construct_partitions(X, fz_type_studied, n_partitions=n_linguistic_variables, categorical_mask=categorical_mask)

Column 'gender' unique classes: ['Male', 'Female']
Column 'region' unique classes: ['Rural', 'Urban']
Column 'income_level' unique classes: ['Middle', 'Low', 'High']
Column 'smoking_status' unique classes: ['Never', 'Past', 'Current']
Column 'alcohol_consumption' unique classes: ['Moderate', 'High']
Column 'physical_activity' unique classes: ['High', 'Moderate', 'Low']
Column 'dietary_habits' unique classes: ['Unhealthy', 'Healthy']
Column 'air_pollution_exposure' unique classes: ['Moderate', 'High', 'Low']
Column 'stress_level' unique classes: ['Moderate', 'High', 'Low']
Column 'EKG_results' unique classes: ['Normal', 'Abnormal']


TypeError: '<' not supported between instances of 'float' and 'str'